# 3.1 Imports

In [1]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

import time
from PIL import Image

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# create function
# open web browser

def configure_driver():
    # incognito window
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # add the argument and make the browser Headless.
    # chrome_options.add_argument("--headless")

    # open web browser
    driver = webdriver.Chrome('/Users/abrahamleung/Documents/chromedriver', options=chrome_options)
    return driver

In [4]:
# load data
df = pd.read_csv('data/activewear.csv')

In [5]:
df.head()

,name,price,img_file,url,num_price,color,apparel
0,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,392.0,BLACK CUT OUT PRINT,BIKINI
1,CK ONE BRALETTE,HKD 392.00,image/CK ONE BRALETTE.png,https://www.calvinklein.com/hk/en/ck-one-brale...,392.0,BLACK CUT OUT PRINT,BRALETTE
2,ACTIVE ICON WOVEN SHORTS,HKD 413.00,image/ACTIVE ICON WOVEN SHORTS.png,https://www.calvinklein.com/hk/en/active-icon-...,413.0,RASPBERRY BLUSH,SHORTS
3,ACTIVE ICON FULL LENGTH ALL-OVER PRINT LEGGINGS,HKD 483.00,image/ACTIVE ICON FULL LENGTH ALL-OVER PRINT L...,https://www.calvinklein.com/hk/en/active-icon-...,483.0,CK BLK CK AOP,LEGGINGS
4,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,903.0,BRIGHT WHITE,JACKET


In [25]:
len(df)

12

# 3.2 Scraping of every link

In [6]:
# testing
n = 0

url = df['url'].iloc[n]
color = df['color'].iloc[n]
apparel = df['apparel'].iloc[n]

In [8]:
url

'https://www.calvinklein.com/hk/en/ck-one-tanga-bikini-KW01435.html?dwvar_KW01435_color=BLACK%20CUT%20OUT%20PRINT'

In [9]:
color

'BLACK CUT OUT PRINT'

In [10]:
apparel

'BIKINI'

In [11]:
# open web browser
driver = configure_driver()

In [12]:
# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

In [13]:
# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

In [18]:
# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath('//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]')
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

In [19]:
# select 'women's fashion' as department
department = driver.find_element_by_xpath('//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]')
department.click()

# may need time sleep
time.sleep(0.1)

In [ ]:
# do not use 'name'
# color created by the color string in html using replace %20
# fashion category created by data cleaning
# use 'women' + color + fashion category

In [20]:
key_words = color + apparel
key_words

'BLACK CUT OUT PRINTBIKINI'

In [21]:
# use 'color' plus 'apparel' as searching key words
key_words = color + apparel

# input search item

search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys(key_words)
search_bar.send_keys(Keys.ENTER)

In [22]:
# get links

links = []
link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
for link in link_paths:
    x = link.get_attribute('href')
    links.append(x)

In [23]:
# no of links can be more than 50
len(links)

48

In [24]:
links[:5]

['https://www.amazon.com/CUPSHE-Womens-Floral-Halter-Swimsuit/dp/B07S223TYL/ref=sr_1_1?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-1',
 'https://www.amazon.com/EJsoyo-Snakeskin-Swimwear-Brazilian-Swimsuits/dp/B0918HCKQT/ref=sr_1_2?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-2',
 'https://www.amazon.com/SHEKINI-Spaghetti-Strappy-Swimsuit-Manhattan/dp/B075P8JS6N/ref=sr_1_3?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-3',
 'https://www.amazon.com/Nulibenna-Swimsuits-Waisted-Swimwear-Bathing/dp/B083J6VHTJ/ref=sr_1_4?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-4',
 'https://www.amazon.com/CUPSHE-Womens-Swimsuit-Strappy-Bathing/dp/B08N146G6S/ref=sr_1_5?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-5']

# 3.3 Scraping of Search Result Details

In [27]:
# testing
# url

URL = links[0]
URL

'https://www.amazon.com/CUPSHE-Womens-Floral-Halter-Swimsuit/dp/B07S223TYL/ref=sr_1_1?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-1'

In [28]:
# input website
driver.get(URL)

In [29]:
# product names
try:
    name = driver.find_elements_by_tag_name('h1')
    name = name[0].text
except:
    name = None

In [30]:
name

"CUPSHE Women's One Piece Swimsuit Cutout Halter Lace Up Twist Bathing Suit"

In [31]:
# product price
try:
    price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
    price = price[0].text
except:
    price = None

In [32]:
price

'HKD 237.32'

In [39]:
# testing
index = 0

In [40]:
# product image
try:
    # size
    img_width,img_height = 300,300

    # get image
    img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
    img = img[0]

    # 'src' = get image source
    src = img.get_attribute('src')

    # download image
    urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

    # resize image (smaller size)
    ori_img = Image.open(f'image/Amazon/{index}.png')
    resize_img = ori_img.resize((img_width,img_height))
    resize_img.save(f'image/Amazon/{index}.png')

    img_file = f'image/Amazon/{index}.png'
except:
    img_file = None


In [41]:
img_file

'image/Amazon/0.png'

In [110]:
# close web browser
driver.close()

In [ ]:
# call the final function as 'main'

In [ ]:
# check if target items displayed by WebDriverWait()
# to save time
try:
    WebDriverWait(driver, 5).until(lambda s: s.find_element_by_id("search-results-category-target").is_displayed())
except TimeoutException:
    return None

In [ ]:
# END